<a href="https://colab.research.google.com/github/ALejandroCh17/Tennis-Ball-Robot/blob/daniel_branch/rally_rover_tennis_ball_detector.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tennis Ball Detection with Convolutional Neural Network

## Importing modules

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sb
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Dropout
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import tensorflow as tf
import os

## Dataset

In [ ]:
# Showing ball image
img = load_img('/kaggle/input/ballsemtpytt/test/balls/ball5.jpg', color_mode='rgb', target_size=(128,128))
img_arr = img_to_array(img)
plt.imshow(img)

In [ ]:
# Showing no ball image
img = load_img('/kaggle/input/ballsemtpytt/test/empty/empty6.jpg', color_mode='rgb', target_size=(128,128))
img_arr = img_to_array(img)
plt.imshow(img)

In [ ]:
def load_images_from_folder(folder):
    images = []
    for filename in os.listdir(folder):
        img = load_img(os.path.join(folder,filename), color_mode='rgb', target_size=(128, 128))
        img_arr = img_to_array(img)
        images.append(img_arr)
    return images

In [ ]:
# Loading images
x_train_balls = load_images_from_folder('/kaggle/input/ballsemtpytt/train/balls')
x_train_empty = load_images_from_folder('/kaggle/input/ballsemtpytt/train/empty')

y_train = np.concatenate( ( np.ones(len(x_train_balls)), np.zeros(len(x_train_empty))) )
x_train = np.concatenate( (np.array(x_train_balls), np.array(x_train_empty) ) )

x_test_balls = load_images_from_folder('/kaggle/input/ballsemtpytt/test/balls')
x_test_empty = load_images_from_folder('/kaggle/input/ballsemtpytt/test/empty')

y_test = np.concatenate( (np.ones(len(x_test_balls)), np.zeros(len(x_test_empty))) )
x_test = np.concatenate( (np.array(x_test_balls), np.array(x_test_empty) ) )

# Shuffling
randomize_train = np.arange(len(y_train))
np.random.shuffle(randomize_train)

randomize_test = np.arange(len(y_test))
np.random.shuffle(randomize_test)

x_train = x_train[randomize_train]/255
y_train = y_train[randomize_train]

x_test = x_test[randomize_test]/255
y_test = y_test[randomize_test]

## Model

In [ ]:
model = Sequential()

model.add(Conv2D(16, kernel_size=3, activation='relu', input_shape=(128,128,3)))
model.add(MaxPooling2D(pool_size = (2,2)))
model.add(Dropout(0.2))
model.add(Conv2D(32, kernel_size=3, activation='relu'))
model.add(MaxPooling2D(pool_size = (2,2)))
model.add(Dropout(0.2))
model.add(Conv2D(64, kernel_size=3, activation='relu'))
model.add(MaxPooling2D(pool_size = (2,2)))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(units=20, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(units=5, activation='relu'))
model.add(Dense(units=1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics = ['accuracy'])

## Training model

In [ ]:
model.fit(x=x_train, y=y_train, validation_data=(x_test, y_test), epochs=50)

## History

In [ ]:
history = pd.DataFrame(model.history.history)
model_loss = history.drop(columns=['accuracy', 'val_accuracy'])
model_accuracy = history.drop(columns=['loss', 'val_loss'])

In [ ]:
model_loss.plot()

In [ ]:
model_accuracy.plot()

## Model evaluation

In [ ]:
predictions = np.rint(model.predict(x_test))

In [ ]:
print( classification_report(y_test, predictions) )

In [ ]:
confusion_matrix(y_test, predictions)

In [ ]:
sample = 43
plt.imshow(x_test[sample], cmap=plt.cm.gray_r)
plt.title('prediction: ' + str(predictions[sample]) )

## Save model

In [ ]:
model.save('model.h5')

## Load model

In [ ]:
model = tf.keras.models.load_model('model.h5')